In [ ]:
!pip install scikit-learn numpy

In [ ]:
%pip install mediapipe opencv-python

In [ ]:
import mediapipe as mp
print(mp.__version__)

0.10.32


In [ ]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from google.colab.patches import cv2_imshow

# Setup model
base_options = python.BaseOptions(model_asset_path="hand_landmarker.task")

options = vision.HandLandmarkerOptions(
    base_options=base_options,
    num_hands=2
)

detector = vision.HandLandmarker.create_from_options(options)

In [ ]:
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cap.release()

if ret:
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    mp_image = mp.Image(
        image_format=mp.ImageFormat.SRGB,
        data=rgb
    )

    result = detector.detect(mp_image)

    print("Hands detected:", len(result.hand_landmarks))

    cv2_imshow(frame)

In [ ]:
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import display, Javascript
from PIL import Image
import io

def capture_image():
    display(Javascript('''
        async function capture() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            await new Promise(resolve => setTimeout(resolve, 2000));

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(track => track.stop());
            div.remove();

            return canvas.toDataURL('image/jpeg');
        }
        capture();
    '''))

    data = eval_js("capture()")
    binary = b64decode(data.split(',')[1])
    return np.array(Image.open(io.BytesIO(binary)))

frame = capture_image()

<IPython.core.display.Javascript object>

In [ ]:
rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

mp_image = mp.Image(
    image_format=mp.ImageFormat.SRGB,
    data=rgb
)

result = detector.detect(mp_image)

if result.hand_landmarks:
    landmarks = []
    for lm in result.hand_landmarks[0]:
        landmarks.extend([lm.x, lm.y, lm.z])

    print("Feature vector length:", len(landmarks))  # Should be 63
else:
    print("No hand detected")

No hand detected
